# Forward Walk Results Visualization

This notebook provides tools for plotting and analyzing forward walk forecasting results.

## 1. Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Configure matplotlib for better plots
plt.style.use('default')
%matplotlib inline

## 2. Define Plotting Function

In [ ]:
def plot_forward_results(csv_file, model_name="Model", save_path=None):
    """
    Plot forward walk results showing actual vs forecasted values
    
    Args:
        csv_file: Path to CSV file with columns: step, actual, forecast, absolute_error
        model_name: Name of the model for plot title
        save_path: Path to save the plot (optional)
    """
    df = pd.read_csv(csv_file)
    
    plt.figure(figsize=(12, 6))
    
    plt.plot(df['step'], df['actual'], 'b-', label='Actual', marker='o', markersize=4)
    plt.plot(df['step'], df['forecast'], 'r--', label='Forecast', marker='s', markersize=4)
    
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.title(f'{model_name} Forward Walk Results: Actual vs Forecast')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")
    
    plt.show()
    
    print(f"Mean Absolute Error: {df['absolute_error'].mean():.6f}")
    print(f"Max Absolute Error: {df['absolute_error'].max():.6f}")
    print(f"Min Absolute Error: {df['absolute_error'].min():.6f}")
    
    return df

## 3. Set Data Paths

In [ ]:
# Define paths to forward walk results
base_dir = "run_forward"

naive_file = os.path.join(base_dir, "naive_walk_forward_results.csv")
nbeats_file = os.path.join(base_dir, "nbeats_walk_forward_results.csv")

# Check if files exist
print(f"Naive results file exists: {os.path.exists(naive_file)}")
print(f"NBEATS results file exists: {os.path.exists(nbeats_file)}")

## 4. Plot NBEATS Forward Walk Results

In [ ]:
# Plot NBEATS results
print("Plotting NBEATS forward walk results...")

if os.path.exists(nbeats_file):
    nbeats_df = plot_forward_results(
        nbeats_file, 
        model_name="NBEATS",
        save_path="nbeats_forward_walk_plot.png"
    )
else:
    print(f"NBEATS results file not found: {nbeats_file}")

## 5. Plot Naive Forward Walk Results

In [ ]:
# Plot Naive results
print("Plotting Naive forward walk results...")

if os.path.exists(naive_file):
    naive_df = plot_forward_results(
        naive_file,
        model_name="Naive", 
        save_path="naive_forward_walk_plot.png"
    )
else:
    print(f"Naive results file not found: {naive_file}")

## 6. Compare Models (Optional)

In [ ]:
# Compare models side by side if both datasets are available
if 'nbeats_df' in locals() and 'naive_df' in locals():
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # NBEATS plot
    ax1.plot(nbeats_df['step'], nbeats_df['actual'], 'b-', label='Actual', marker='o', markersize=3)
    ax1.plot(nbeats_df['step'], nbeats_df['forecast'], 'r--', label='Forecast', marker='s', markersize=3)
    ax1.set_title('NBEATS: Actual vs Forecast')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylabel('Value')
    
    # Naive plot
    ax2.plot(naive_df['step'], naive_df['actual'], 'b-', label='Actual', marker='o', markersize=3)
    ax2.plot(naive_df['step'], naive_df['forecast'], 'r--', label='Forecast', marker='s', markersize=3)
    ax2.set_title('Naive: Actual vs Forecast')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Value')
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print comparison statistics
    print("\n=== Model Comparison ===")
    print(f"NBEATS MAE: {nbeats_df['absolute_error'].mean():.6f}")
    print(f"Naive MAE:  {naive_df['absolute_error'].mean():.6f}")
    
    mae_improvement = ((naive_df['absolute_error'].mean() - nbeats_df['absolute_error'].mean()) / 
                       naive_df['absolute_error'].mean()) * 100
    print(f"NBEATS improvement over Naive: {mae_improvement:.2f}%")
else:
    print("Comparison requires both model results to be loaded.")

## 7. Error Analysis

In [ ]:
# Detailed error analysis for available models
models_data = {}

if 'nbeats_df' in locals():
    models_data['NBEATS'] = nbeats_df
if 'naive_df' in locals():
    models_data['Naive'] = naive_df

if models_data:
    # Plot error distributions
    fig, axes = plt.subplots(1, len(models_data), figsize=(6*len(models_data), 5))
    if len(models_data) == 1:
        axes = [axes]  # Make it iterable for single model
    
    for i, (model_name, df) in enumerate(models_data.items()):
        axes[i].hist(df['absolute_error'], bins=20, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{model_name} - Error Distribution')
        axes[i].set_xlabel('Absolute Error')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        # Add statistics text
        mean_err = df['absolute_error'].mean()
        std_err = df['absolute_error'].std()
        axes[i].axvline(mean_err, color='red', linestyle='--', 
                       label=f'Mean: {mean_err:.4f}')
        axes[i].legend()
    
    plt.tight_layout()
    plt.savefig('error_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Summary statistics table
    print("\n=== Error Statistics Summary ===")
    summary_data = []
    for model_name, df in models_data.items():
        summary_data.append({
            'Model': model_name,
            'Mean Error': f"{df['absolute_error'].mean():.6f}",
            'Std Error': f"{df['absolute_error'].std():.6f}",
            'Max Error': f"{df['absolute_error'].max():.6f}",
            'Min Error': f"{df['absolute_error'].min():.6f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
else:
    print("No model data available for error analysis.")

## 8. Custom Plotting (Optional)

Use this cell to create custom plots or analyze specific aspects of your results.

In [ ]:
# Example: Plot just a subset of time steps
# Modify this cell as needed for your specific analysis

# Uncomment and modify the code below for custom analysis:

# if 'nbeats_df' in locals():
#     # Plot only first 50 time steps
#     subset_df = nbeats_df.head(50)
#     
#     plt.figure(figsize=(10, 5))
#     plt.plot(subset_df['step'], subset_df['actual'], 'b-', label='Actual', marker='o')
#     plt.plot(subset_df['step'], subset_df['forecast'], 'r--', label='Forecast', marker='s')
#     plt.title('NBEATS Results (First 50 Steps)')
#     plt.xlabel('Time Step')
#     plt.ylabel('Value')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.show()

print("Add your custom plotting code in this cell.")